In [34]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Food -> Nutrients

The item to look up in the USDA archive

In [41]:
amazon_url = 'https://www.amazon.com/Organic-Valley-Boxes-Chocolate-Ounces/dp/B00WVSNIEY/ref=sr_1_2_sspa?ie=UTF8&qid=1532194224&sr=8-2-spons&keywords=organic+valley+lowfat+milk&psc=1'
nutrient = 'Vitamin A'

Translate into query url

In [42]:
# open Selenium webdriver in headless mode
chrome_options = Options()
chrome_options.add_argument("headless")
driver = webdriver.Chrome(chrome_options=chrome_options)

# get the 'item title' html element from the amazon item page
driver.get(amazon_url)
product_title = driver.find_element_by_xpath('//*[@id="productTitle"]').text

# parse the product title to a smarter lookup query
# TODO: implement this
product_title = product_title.replace(',', '').replace('%', '')

# amazon url to lookup
lookup = product_title 

# the query url
query_url = 'https://ndb.nal.usda.gov/ndb/search/list?SYNCHRONIZER_TOKEN=22d50fc9-d900-4447-afa8-cda5f67ea1e5&SYNCHRONIZER_URI=%2Fndb%2Fsearch%2Flist&qt=&qlookup={}'.format(lookup)

Extract the NBD id of the best hit of the query

In [43]:
# go to the page
driver.get(query_url)

# find the first nbd_id in the query
nbd_id = driver.find_element_by_xpath('//*[@id="quickform"]/div[4]/table/tbody/tr[1]/td[2]/a').text

Use the NBD id to get the full nutrient report from the USDA website

In [44]:
import json
api_url = "https://api.nal.usda.gov/ndb/reports/?ndbno={}&type=f&format=json&api_key=2RKrTS6ad7iBE5RapN91xfUguREfFsebWNnDer1X".format(nbd_id)
print(api_url)
# submit query
driver.get(api_url)

# get the response
soup = BeautifulSoup(driver.page_source, "lxml")
json_response = json.loads(soup.find("body").text)

https://api.nal.usda.gov/ndb/reports/?ndbno=45289703&type=f&format=json&api_key=2RKrTS6ad7iBE5RapN91xfUguREfFsebWNnDer1X


In [45]:
json_response

{'report': {'food': {'ds': 'Label Insight',
   'ing': {'desc': 'LOWFAT MILK, SUGAR, COCOA (PROCESSED WITH ALKALI), SALT, CARRAGEENAN, VITAMIN A PALMITATE, VITAMIN D3.',
    'upd': '07/14/2017'},
   'manu': 'Super Store Industries',
   'name': '1% MILK FAT CHOCOLATE LOW FAT MILK, UPC: 717544205074',
   'ndbno': '45289703',
   'nutrients': [{'derivation': 'LCCS',
     'group': 'Proximates',
     'measures': [{'eqv': 240.0,
       'eunit': 'ml',
       'label': 'cup',
       'qty': 1.0,
       'value': '161'}],
     'name': 'Energy',
     'nutrient_id': '208',
     'unit': 'kcal',
     'value': '67'},
    {'derivation': 'LCCS',
     'group': 'Proximates',
     'measures': [{'eqv': 240.0,
       'eunit': 'ml',
       'label': 'cup',
       'qty': 1.0,
       'value': '9.00'}],
     'name': 'Protein',
     'nutrient_id': '203',
     'unit': 'g',
     'value': '3.75'},
    {'derivation': 'LCCS',
     'group': 'Proximates',
     'measures': [{'eqv': 240.0,
       'eunit': 'ml',
       'label'

In [39]:
%%time
for ntr in json_response['report']['food']['nutrients']:
    if nutrient in ntr['name']:
        print(ntr['value'])
        if float(ntr['value']) > 0:
            print('The nutrient is present.')
        else:
            print('The nutrient is absent.')

208
The nutrient is present.
CPU times: user 636 µs, sys: 714 µs, total: 1.35 ms
Wall time: 726 µs
